<a href="https://colab.research.google.com/github/Ilyes-Azni/Ilyes-Azni/blob/main/Book_CV_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Purpose 🥇

In addition to the model that we will propose to predict some books ratings, OCR can boost the project and enable some human machine interactions for example filming or taking a picture of a book and give the rating out of the picture (thanks to computer vision yay! 👓)



## How

To perform OCR on books I guessed that the human eye focuses on certain parts of a front cover where there are informations. For sure, we read the title then the author or the other way around. Because of this, I thought that leveraging yolo and perform some finetuning on examples that highlights this human process could achieve some performance. When the machine is able to locate, we then want it to be able to "read". This is where pytesseract comes in handy! PT is a library, making the use of an OCR model the easy way. We could have done a custom one, but before jumping in, a quick POC is important. If the POC is cool enough I'll make my own OCR for the project!

# Model Training of YOLOv5 on books bounding boxes

In [1]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [8]:
# Train the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 50 --data /content/drive/MyDrive/Data/DATA/data.yaml --weights yolov5s.pt

2024-07-22 10:10:07.881172: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-22 10:10:07.881221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 10:10:07.883276: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/Data/DATA/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_sca

# Performing OCR on new images

In [16]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,915 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [17]:
!pip install pytesseract

In [24]:
path_img = "/content/51evTcoKMGL.jpg"

In [25]:
import torch
import cv2
import pytesseract
import matplotlib.pyplot as plt


model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp3/weights/best.pt')


image = cv2.imread(path_img)


results = model(image)


df = results.pandas().xyxy[0]



for index, row in df.iterrows():
    xmin, ymin, xmax, ymax, confidence, class_id, name = row
    # Draw bounding box
    cv2.rectangle(image, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)

    # Extract ROI
    roi = image[int(ymin):int(ymax), int(xmin):int(xmax)]

    # Convert ROI to grayscale for better OCR results
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Perform OCR on the ROI
    ocr_result = pytesseract.image_to_string(roi_gray)

    # Print OCR results
    print(f'Recognized text: {ocr_result.strip()}')

    # Optionally, draw OCR results on the image
    cv2.putText(image, ocr_result.strip(), (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Convert BGR image to RGB for display
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image with bounding boxes and OCR results
plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off')
plt.show()



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-22 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Recognized text: gO

Great
Expectations
iCHARLES DICKENS


# Code for live recognition using webcam

In [27]:
# Initialize webcam capture (0 is typically the default camera)
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)
    df = results.pandas().xyxy[0]

    # Loop over detected objects
    for index, row in df.iterrows():
        xmin, ymin, xmax, ymax, confidence, class_id, name = row

        # Draw bounding box
        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)

        # Extract ROI
        roi = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

        # Convert ROI to grayscale for better OCR results
        roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Perform OCR on the ROI
        ocr_result = pytesseract.image_to_string(roi_gray)

        # Print OCR results
        print(f'Recognized text: {ocr_result.strip()}')

        # Optionally, draw OCR results on the image
        cv2.putText(frame, ocr_result.strip(), (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Live OCR', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()